In [ ]:
from easyroutine.utils import path_to_parents
path_to_parents(1)
%load_ext autoreload
%autoreload 2

Changed working directory to: /orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine


In [2]:
from easyroutine.interpretability import HookedModel
from easyroutine.interpretability.tools import LogitLens




/u/dssc/francescortu/VisualComp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/dssc/francescortu/VisualComp/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
model = HookedModel.from_pretrained("hf-internal-testing/tiny-random-LlamaForCausalLM")

/u/dssc/francescortu/VisualComp/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


[02/13/25 15:47:39] INFO     2025-02-13 15:47:39,839 - HookedModel_stdout - INFO - Model loaded in 1  ]8;id=760429;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py\logger.py]8;;\:]8;id=330394;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py#122\122]8;;\
                             devices. First device: cuda:0                                                         

                    INFO     2025-02-13 15:47:39,845 - HookedModel_stdout - INFO -                    ]8;id=346375;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py\logger.py]8;;\:]8;id=164228;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py#122\122]8;;\
                                                         The model is using the custom eager                       
                             attention implementation that support attention matrix hooks because I                
                             get config.attn_impelemntation == 'custom_eager'. If you don't want                   
                             this, you can call HookedModel.restore_original_modules.                              
                                                         However, we reccomend using this                          
                             implementation since the base one do not contains attention matrix hook               
                             resulting in unexpected behaviours.                                                   
                                                                                                                   

                    INFO     2025-02-13 15:47:39,848 - HookedModel_stdout - INFO - Setting custom     ]8;id=360975;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py\logger.py]8;;\:]8;id=754820;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py#122\122]8;;\
                             modules.                                                                              

In [4]:
logit_lens = LogitLens.from_model(model)

In [5]:
import torch
from easyroutine.interpretability import ExtractionConfig

fake_dataset = [
    {
        "input_ids": torch.randint(0, 100, (1, 100)),
        "attention_mask": torch.randint(0, 2, (1, 100)),
    },
    {
        "input_ids": torch.randint(0, 100, (1, 100)),
        "attention_mask": torch.randint(0, 2, (1, 100)),
    }
]

cache = model.extract_cache(
    fake_dataset,
    target_token_positions=["last"],
    extraction_config = ExtractionConfig(
        extract_resid_out=True
    )
)

                    INFO     2025-02-13 15:47:39,912 - HookedModel_stdout - INFO - Extracting cache   ]8;id=524725;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py\logger.py]8;;\:]8;id=796928;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py#122\122]8;;\

                    INFO     2025-02-13 15:47:39,914 - HookedModel_stdout - INFO - Forward pass       ]8;id=445440;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py\logger.py]8;;\:]8;id=548334;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py#122\122]8;;\
                             started                                                                               

Extracting cache::   0%|          | 0/2 [00:00<?, ?it/s]

Extracting cache:: 100%|██████████| 2/2 [00:00<00:00,  2.54it/s]


[02/13/25 15:47:40] INFO     2025-02-13 15:47:40,707 - HookedModel_stdout - INFO - Forward pass       ]8;id=933108;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py\logger.py]8;;\:]8;id=906604;file:///orfeo/cephfs/home/dssc/francescortu/VisualComp/easyroutine/easyroutine/logger.py#122\122]8;;\
                             finished - started to aggregate different batch                                       

In [8]:
cache

ActivationCache(`resid_out_0, resid_out_1, logits, mapping_index, example_dict`)

In [18]:
out = logit_lens.compute(
    activations=cache,
    target_key="resid_out_{i}"
)

Computing Logit Lens of resid_out_{i}: 100%|██████████| 2/2 [00:33<00:00, 16.51s/it]


In [19]:
out["logit_lens_resid_out_0"].shape

torch.Size([2, 1, 32000])